In [1]:
import tensorflow as tf
import re

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# create config to use both gpus on wickerman machine:
# 1. Nvidia A40 (25GB memory allocation)
# 2. Nvidia RTX 3060 (11GB memory allocation)

gpus = tf.config.list_physical_devices('GPU')

# tf.config.set_visible_devices(gpus[0], 'GPU') # use Use Nvidia A40 only

# log_dev_conf_a40 = tf.config.LogicalDeviceConfiguration(
#     memory_limit=40*1024 # 25 GB allocation for a40 GPU
# )

# tf.config.set_logical_device_configuration(
#     gpus[0], # select GPU_0, i.e., Nvidia A40
#     [log_dev_conf_a40] # apply 25GB config
# )

tf.config.set_visible_devices(gpus[1], 'GPU') # use RTX3060 only

log_dev_conf_rtx3060 = tf.config.LogicalDeviceConfiguration(
    memory_limit=6*1024 # 11 GB allocation for rtx3060 GPU
)

tf.config.set_logical_device_configuration(
    gpus[1], # select GPU_1, i.e., Nvidia rtx3060
    [log_dev_conf_rtx3060] # apply 11GB config
)

# MultiGPU setup
# Create a MirroredStrategy.
strategy = None
# strategy = tf.distribute.MirroredStrategy()
# print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")

2024-08-16 17:10:46.935230: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 17:10:46.935253: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 17:10:46.935874: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 17:10:46.939617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-16 17:10:47.530726: W tensorflow/compiler/tf2

2 Physical GPUs, 1 Logical GPU


2024-08-16 17:10:48.309958: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-16 17:10:48.311876: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-16 17:10:48.333285: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
import json
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.utils import Progbar
from tensorflow.keras.metrics import Mean
from pqdm.threads import pqdm
from tqdm import tqdm

In [3]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

model_id = "google/flan-t5-small"

# with strategy.scope():
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = TFT5ForConditionalGeneration.from_pretrained(model_id)
    # tokenizer = AutoTokenizer.from_pretrained("models/v1")
    # model = TFT5ForConditionalGeneration.from_pretrained("models/v1")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

2024-08-16 17:11:52.974994: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1101] failed to allocate 6.00GiB (6442450944 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-08-16 17:11:52.975189: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1101] failed to allocate 5.40GiB (5798205440 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-08-16 17:11:52.975369: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1101] failed to allocate 4.86GiB (5218384896 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
/home/vedant/miniconda3/envs/tf/lib/python3.9/site-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
